# Setup

In [ ]:
LOCAL = True

LOCAL_HOST = '192.168.50.200'

In [ ]:
import requests
import json
from IPython.display import display

if LOCAL:
    SITE_URL_BASE = f"""http://{LOCAL_HOST}:5173/"""
else:
    SITE_URL_BASE = """https://eeva.site/"""

def response_or_error(response):
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
    else:
        return response.json()
    
def get(path: str) -> dict:
    return response_or_error(requests.get(f"{SITE_URL_BASE}{path}"))

def post(path: str, body: dict | str) -> dict:
    if isinstance(body, str):
        body = json.loads(body)
    return response_or_error(requests.post(f"{SITE_URL_BASE}{path}", json=body))

def put(path: str, body: dict | str) -> dict:
    if isinstance(body, str):
        body = json.loads(body)
    return response_or_error(requests.put(f"{SITE_URL_BASE}{path}", json=body))

def delete(path: str) -> dict:
    return response_or_error(requests.delete(f"{SITE_URL_BASE}{path}"))

# Tools

### View Questions

In [ ]:
questions = get("api/questions")
questions

### Update questions

In [ ]:
questions_json_path = "data/questions.json"

with open(questions_json_path, "r", encoding="utf-8") as file:
    questions = json.load(file)
    display(put("api/questions", questions))

### View forms

In [ ]:
forms = get("api/forms")
forms

### Create/update form

In [ ]:
form_id = "default"
question_ids = ["lytte-tale", "issmag-er-du", 'are-you-morning-person-DK', 'bukke-under-for-gruppepres']

put(f"api/forms/{form_id}", {"questions": question_ids})

### View responses

In [ ]:
form_responses = get("api/form-responses")
form_responses